## From ISA JSON to RDF Linked Data with ISALDSerializer function:

### Getting ISAtools and importing the latest module for conversion to JSON-LD from ISA-JSON

### Abstract:

The goal of this tutorial is to show how to go from an ISA document to an equivalent RDF representation using python tools but also to highlight some of the limitations of existing libraries and point to alternative options to complete a meainingful conversion to RDF Turtle format.

This notebook mainly highlights the new functionality coming with ISA-API rc10.3 latest release which allows to convert ISA-JSON to ISA-JSON-LD, with the choice of 3 popular ontological frameworks for semantic anchoring. These are:
- [obofoundry](http://www.obofoundry.org), a set of interoperable ontologies for the biological domain.
- [schema.org](https://schema.org), the search engine orientated ontology developed by companies such as Yandex,Bing,Google 
- [wikidata](https://wikidata.org), a set of semantic concepts backing wikipedia and wikidata resources.

These frameworks have been chosen for interoperability.


This notebook has a companion notebook which goes over the exploration of the resulting RDF representations using a set of SPARQL queries.
Check it out [here](http://localhost:8888/notebooks/isa-cookbook/content/notebooks/isa-jsonld%20exploration%20with%20SPARQL.ipynb)


In [ ]:
import os
import json
from json import load
import datetime
import isatools

from isatools.convert.json2jsonld import ISALDSerializer

#### Loading an ISA-JSON document in memory with json.load() function

Prior to invoking the ISALDserializer function, we need to do 3 things.
* First, pass a url or a path to the ISA-JSON instance to convert to JSON-LD
* Second, select the ontology framework used for the semantic conversion. One may choose from the following 3 options:
    - obofoundry ontologies, abbreviated as `obo`
    - schema.org ontology, abbreviated as `sdo`
    - wikidata ontology, abbreviated as `wdt`
* Third, choose if to rely on embedding the @context file in the output or relying on url to individual contexts. By default, the converter will embedded the 'all in one' context information. The reason for this is the lack of support for JSON-LD 1.1 specifications in many of the python libraries supported RDF parsing (e.g. rdflib)


In [ ]:
instance_path = os.path.join("./output/BII-S-3-synth/", "isa-new_ids.json")

with open(instance_path, 'r') as instance_file:
        instance = load(instance_file)
        instance_file.close()
        
ontology = "wdt"

In [ ]:
# we now invoke the ISALDSerializer function

serializer = ISALDSerializer(instance)
serializer.set_ontology(ontology)
serializer.set_instance(instance)

jsonldcontent = serializer.output

Now that the conversion is performed, we can write the resulting ISA-JSON-LD to file:

In [ ]:
isa_json_ld_path = os.path.join("./output/BII-S-3-synth/", "isa-new_ids-BII-S-3-ld-" + ontology + "-v1.json")
with open(isa_json_ld_path, 'w') as outfile:
    json.dump(jsonldcontent,  outfile, ensure_ascii=False, indent=4)

### Let's now try converting an ISA-JSONLD instance to RDF turtle using pyLD  Python library

In [ ]:
import pyld
import os
import isatools
import json

from pyld import *

Reading ISA JSONLD in memory first:

In [ ]:
with open(isa_json_ld_path,'r') as LDin:
    json_doc = json.load(LDin) 

Converting JSONLD to **N-Quads** using the `to_rdf` function afforded by the **`pyLD`** package

In [ ]:
rdf_as_nq = pyld.jsonld.to_rdf(json_doc, {'format': 'application/n-quads', 'useRdfType': "True"})

```warning```

- It is important to know that  **`pyLD`** does not support RDF Turtle serialization yet and that python **`rdflib`** and its **`rdflib_jsonld`** do not support JSONLD 1.1 specifications, which is exploited by the ISA-JSONLD conversion (for example, the need to use @nest element).



- Serializing to file the n-quads representation of the ISA-JSONLD.

In [ ]:
isa_nquads_path = os.path.join("./output/BII-S-3-synth/", "isa-new_ids-BII-S-3-ld-" + ontology + "-v1.nq")

with open(isa_nquads_path,'w') as NQout:
    NQout.write(rdf_as_nq)

**IMPORTANT**: this conversion is currently incomplete with this python component. 
the IS JSON-LD can be fully converted with JSON-LD playground site and easyrdf/converter tools. 


### Packaging the ISA archive and its various serializations (ISA-Tab, ISA-JSON, ISA-JSON-LD) as a Research Object Crate 

In [ ]:
from rocrate.rocrate import ROCrate
from rocrate.model.person import Person
from rocrate.model.dataset import Dataset
from rocrate import rocrate_api

#### Instantiating a Research Object and providing basic metadata

In [ ]:
a_crate_for_isa = ROCrate()

a_crate_for_isa.name = "ISA JSON-LD representation of BII-S-3"
a_crate_for_isa.description = "ISA study serialized as JSON-LD using " + ontology + " ontology mapping"
a_crate_for_isa.keywords = ["ISA", "JSON-LD"]
a_crate_for_isa.license = "https://creativecommons.org/licenses/by/4.0/"
a_crate_for_isa.creator = Person(a_crate_for_isa,"https://www.orcid.org/0000-0001-9853-5668")

#### Adding the two ISA RDF serializations to the newly created Research Object

In [ ]:
files = [isa_json_ld_path,isa_nquads_path ]
# with a python comprehension, we do it like this:
[a_crate_for_isa.add_file(file) for file in files]

Now adding a dataset to the Research Object, which is meant to describe a bag of associated images.

In [ ]:
ds = Dataset(a_crate_for_isa, "image")
a_crate_for_isa.add(ds)

#### Finally, writing the Research Object to file

In [ ]:
ro_outpath = "./output/BII-S-3-synth/ISA_in_a_ROcrate"
a_crate_for_isa.write_crate(ro_outpath)

In [ ]:
#### Alternately, a zipped archive can be created as follows:
a_crate_for_isa.write_zip(ro_outpath)

## About this notebook

- authors: Philippe Rocca-Serra (philippe.rocca-serra@oerc.ox.ac.uk), Dominique Batista (dominique.batista@oerc.ox.ac.uk)
- license: CC-BY 4.0
- support: isatools@googlegroups.com
- issue tracker: https://github.com/ISA-tools/isa-api/issues
